# <b><center> Social Networks EDA </b></center>


## Entendimiento de los Datos:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import re
from difflib import SequenceMatcher

## <u>Leer los datos que en el nombre contienen un mes:</u></br>
### <b>Instagram:</b>

Cargar los 12 archivos que en el nombre tienen un mes, en 3  DataFrames, uno de cada red social, donde cada uno de los cuales debe contener las siguientes columnas:

Instagram:
* Username
* Name
* Subscribers o Followers
* Country
* Authentic
* Engagement
* Category1
* Category2
* Month

In [ ]:
ig_june = (
    pd.read_csv('DATA/instagramjune2022.csv')
    .assign(Month='June')
    .rename(columns={
        "instagram name": "username",
        "influencer name ":"name",
        "Category_1": "category1",
        "Category_2": "category2",
        "Subscribers count": "followers",
        "Views avg.": "country",
        "Likes avg": "authentic",
        "Comments avg.": "average",
        "Month": "month"})
)
ig_june.country = ig_june.country.apply(lambda x: np.nan if re.search(r'\d', str(x)) else str(x))

In [ ]:
ig_sep = (
    pd.read_csv('DATA/instagramsep2022.csv')
    .assign(Month='September')
    .rename(columns={
        "Instagram name": "username",
        " Name": "name",
        "Subscribers": "followers",
        "Audience country": "country",
        "Authentic engagement\n": "authentic",
        "Engagement average\r\n": "average",
        "Category_1": "category1",
        "Category_2": "category2",
        "Month": "month"})
    .drop('S.no', axis=1)
)

In [ ]:
ig_nov = (
    pd.read_csv('DATA/instagramNov2022.csv')
    .assign(Month='November')
    .drop('s.no', axis=1)
    .rename(columns={
        "Name": "username",
        "Instagram Name": "name",
        "Category-1": "category1",
        "Followers": "followers",
        "\nCountry": "country",
        "Eng. (Auth.)": "authentic",
        "Eng. (Avg.)": "average",
        "Category-2": "category2",
        "Month": "month"})
)

In [ ]:
ig_dec = (
    pd.read_csv('DATA/INSTAGRAMDEC2022.csv')
    .assign(Month='December')
    .rename(columns={
        "name": "username",
        "instagram name": "name",
        "Category_1": "category1",
        "Category_2": "category2",
        "Eng. (Auth.)": "authentic",
        "Eng. (Avg.)": "average",
        "Month": "month"})
)

In [ ]:
ig_june.isna().sum()

Definiendo una función que transforma los números de texto a int en todo una columna

In [ ]:
def cientific_int(column: pd.Series) -> pd.Series:
    return (
        column
        .fillna('0')
        .replace({'K': '*1e3', 'M': '*1e6', 'G':'*1e9'}, regex=True)
        .map(pd.eval)
        .astype(int)
        .replace(0, np.nan)
    )

numeric_text_cols = ['followers', 'average', 'authentic']

In [ ]:
ig_june.loc[:, numeric_text_cols] = ig_june[numeric_text_cols].apply(cientific_int)
ig_sep.loc[:, numeric_text_cols] = ig_sep[numeric_text_cols].apply(cientific_int)
ig_nov.loc[:, numeric_text_cols] = ig_nov[numeric_text_cols].apply(cientific_int)
ig_dec.loc[:, numeric_text_cols] = ig_dec[numeric_text_cols].apply(cientific_int)

ig_months es la combinación de todos los meses, eliminando aquellos registros duplicados.
Tener cuidado con los datos numéricos, ya que son datos incompletos. Para sacar promedios se tiene que considerar los df por separado

In [ ]:
ig_months = (
    pd.concat([ig_june, ig_sep, ig_nov, ig_dec], ignore_index=True)
    .drop("Rank", axis=1)
    .drop_duplicates()
)

Verificando duplicados

In [ ]:
ig_months.duplicated().sum()

# Análisis instagram

### Número total de cuentas distintas en instagram: 2409

In [ ]:
ig_months.username.drop_duplicates().size

### Media de seguidores
Calcular el promedio de seguidores, authentic, engagement, de cada cuenta. Es decir, cada red social cuenta con 4 archivos de 4 meses distintos, si una cuenta aparece en los 4 meses, debera calcular el promedio de esos 4 meses, si una cuenta solo aparece 2 meses, se hara el promedio de esos dos meses y asi sucesivamente. SI el valor es nulo, vacio, N/A, o 0 no se debe promediar.

In [ ]:
ig_means = (
    ig_months
    .groupby('username')
     # aggregate functiosn skips nan by deffault
    .aggregate({
        'followers': 'mean',
        'authentic': 'mean',
        'average': 'mean'
    })    
    .reset_index()
    .sort_values('followers', ascending=False)
)

### Metricas de Valor

Metricas de Valor de cada cuenta. Para este punto debe utilizar los promedios del punto anterior: Promedio Engagement/Promedio Followers.

In [ ]:
ig_means['engagement_ratio'] = ig_means.average / ig_means.followers

In [ ]:
ig_means

### Graficar iniciales

Graficar en un histograma el numero de cuentas que inicien por cada letra del alfabeto (Mayusculas y minusculas son tomadas de igual forma) y agrupar en una sola categoria todos los que no comiencen con una letra del alfabeto.

In [ ]:
def get_inicial(name: str) -> str:
    name = str(name)
    first_is_letter = re.search(r'^[a-zA-Z]', name)
    if first_is_letter:
        return name[0].upper()
    else:
        return 'other'

In [ ]:
initials = (
    ig_months.name
    .map(get_inicial)
    .value_counts()
    .reset_index(name='frequency')
    .rename(columns={
        'index': 'letter',
        'name': 'letter'
    })
)
plt.figure(figsize=(10, 6)) 
plt.bar(initials.letter, initials.frequency,  color="cadetblue")
plt.show()

### ¿Cual es la cuenta de IG con mas caracteres en su nombre?

In [ ]:
name_len = (
    ig_months[['name', 'username']]
    .assign(len_name = ig_months.name.map(lambda x: len(str(x))))
    .assign(len_username = ig_months.username.map(lambda x: len(str(x))))
)
ids1 = name_len.loc[name_len.len_name.idxmax()]
ids2 = name_len.loc[name_len.len_username.idxmax()]

**username**

In [ ]:
ids2

**name**

In [ ]:
ids1

### ¿Cuantas cuentas de Instagram tienen ya sea en su username o name las letras de las iniciales del primer nombre de cada uno de los integrantes de tu equipo?

In [ ]:
(ig_months.name.str.contains(r'[GgIiMm]') | ig_months.name.str.contains(r'[GgIiMm]')).sum()

### Defina una métrica que identifique las 10 cuentas mas importantes de Instagram

`engagement_size` es una metrica que toma en cuenta el engagement autentico y la cantidad de followers

In [ ]:
ig_means = (
    ig_means
    .assign(
        engagement_size = (ig_means.authentic / ig_means.followers) * (ig_means.followers / max(ig_means.followers))
    )
    .sort_values(['engagement_size'], ascending=False)
)
ig_means[['username', 'engagement_size']].head(10)

### Grafique un histograma del numero de seguidores promedio por pais.

In [ ]:
ig_country_means = (
    ig_means
    .merge(ig_months, on='username', how='left')
    [['username', 'country', 'followers_x']]
    .groupby('country')
    .aggregate({'followers_x': 'mean'})
    .reset_index()
    .rename(columns={
        'followers_x': 'mean_followers'
    })
    .sort_values('mean_followers', ascending=False)
)
ig_country_means
plt.bar(ig_country_means.country, ig_country_means.mean_followers)
plt.show()# 

### Cree una serie con todas las categorias existentes en todos los archivos de Instagram.

In [ ]:
ig_categories = (ig_months.category1 + ig_months.category2).dropna().drop_duplicates().reset_index(name='category')
ig_categories = list(ig_categories.category)

In [ ]:
category_re = r'[A-Z][a-z\s]+[a-z]$'
res = set()
for element in ig_categories:
    has_categories = re.findall(category_re, str(element))
    if has_categories:
        res = res.union(set(has_categories))

ig_categories = pd.Series(list(res))
ig_categories

### Cree una serie con todos los paises que encuentre en los archivos.

In [ ]:
all_countries = ig_months.country.dropna().drop_duplicates()
all_countries

### Grafique en un histograma el numero de seguidores por categoria.

In [ ]:
# ig_category_followers = (
#     all_categories
#     .merge(ig_months, on='categorie', how='left')
#     [['username', 'country', 'followers_x']]
#     .groupby('country')
#     .mean('followers_x')
#     .reset_index()
#     .rename(columns={
#         'followers_x': 'mean_followers'
#     })
#     .sort_values('mean_followers', ascending=False)
# )
# plt.bar(ig_country_means.country, ig_country_means.mean_followers)
# plt.show()

## <u>Leer los datos que en el nombre no contienen mes (resumen):</u></br>

In [ ]:
ig_resumen = pd.read_csv('DATA/instagram.csv')
ig_resumen

In [35]:
tiktok_resumen = pd.read_csv('DATA/tiktok.csv')
tiktok_resumen

In [36]:
youtube_resumen = pd.read_csv('DATA/youtube.csv')
youtube_resumen